In [247]:
import pandas as pd

df = pd.read_csv("/content/train_preprocessed.csv")
text = df['comment_text'].values[:150000]

In [248]:
import torch
from sonar.inference_pipelines.text import TextToEmbeddingModelPipeline

embedder = TextToEmbeddingModelPipeline(
  encoder="text_sonar_basic_encoder",
  tokenizer="text_sonar_basic_encoder",
  device=torch.device("cuda"),
  dtype=torch.float16,
)

In [249]:
with torch.inference_mode():
  embeddings = embedder.predict(text, source_lang="eng_Latn")

print(embeddings.shape)

/usr/local/lib/python3.11/dist-packages/sonar/inference_pipelines/text.py:259: UserWarning: For 1 input tensors for SONAR text encoder, the length was truncated to 514 elements.
  warnings.warn(


torch.Size([50, 1024])


In [250]:
from sonar.inference_pipelines.text import EmbeddingToTextModelPipeline

decoder = EmbeddingToTextModelPipeline(
    decoder="text_sonar_basic_decoder",
    tokenizer="text_sonar_basic_encoder",
    device=torch.device("cuda:0"),
    dtype=torch.float16,
)

In [251]:
with torch.inference_mode():
  text_decoded = decoder.predict(embeddings, target_lang="eng_Latn")

In [253]:
id = 10
print("Texto original:", text[id])
print("Texto reconstruído:", text_decoded[id])

Texto original:   fair use rationale for image wonju jpg  thanks for uploading image wonju jpg  i notice the image page specifies that the image is being used under fair use but there is no explanation or rationale as to why its use in wikipedia articles constitutes fair use  in addition to the boilerplate fair use template  you must also write out on the image description page a specific explanation or rationale for why using this image in each article is consistent with fair use please go to the image description page and edit it to include a fair use rationale if you have uploaded other fair use media  consider checking that you have specified the fair use rationale on those pages too  you can find a list of  image  pages you have edited by clicking on the my contributions link  it is located at the very top of any wikipedia page when you are logged in and then selecting image from the dropdown box  note that any fair use images uploaded after may and lacking such an explanation wil

# Desintoxicação boba

In [ ]:
import numpy as np

# Adiciona os embeddings ao dataframe
df_embed = df.iloc[:len(embeddings)].copy()
df_embed['embedding'] = [emb.detach().cpu().numpy() for emb in embeddings]

# Define categorias para calcular
categorias = ['identity_hate', 'insult', 'obscene', 'severe_toxic', 'threat', 'toxic']
vetores_categoria = {}

# Calcula vetores médios
for cat in categorias:
    subset = df_embed[df_embed[cat] == 1]
    if not subset.empty:
        vetor_medio = np.mean(np.stack(subset['embedding'].values), axis=0)
        vetores_categoria[cat] = vetor_medio
        print(f"{cat}: vetor médio calculado com {len(subset)} exemplos.")
    else:
        print(f"{cat}: nenhum exemplo encontrado.")

# Também podemos calcular o vetor médio de comentários 'neutros'
neutros = df_embed[(df_embed[categorias].sum(axis=1) == 0)]
vetor_neutro = np.mean(np.stack(neutros['embedding'].values), axis=0)
vetores_categoria['neutro'] = vetor_neutro
print(f"neutro: vetor médio calculado com {len(neutros)} exemplos.")

identity_hate: vetor médio calculado com 1318 exemplos.
insult: vetor médio calculado com 7410 exemplos.
obscene: vetor médio calculado com 7960 exemplos.
severe_toxic: vetor médio calculado com 1514 exemplos.
threat: vetor médio calculado com 445 exemplos.
toxic: vetor médio calculado com 14356 exemplos.
neutro: vetor médio calculado com 134758 exemplos.


In [227]:
# Define categorias tóxicas
categorias = ['identity_hate', 'insult', 'obscene', 'severe_toxic', 'threat', 'toxic']

# Filtra os comentários tóxicos
comentarios_toxicos = df_embed[df_embed[categorias].sum(axis=1) > 0].copy()

# Para cada linha, identifica as categorias ativas
comentarios_toxicos['categorias_ativas'] = comentarios_toxicos[categorias].apply(
    lambda row: [cat for cat in categorias if row[cat] == 1], axis=1
)

# Exibe uma amostra dos primeiros N comentários tóxicos (ajuste o número se quiser)
N = 10
for idx, row in comentarios_toxicos.head(N).iterrows():
    print(f"Índice: {idx}")
    print(f"Texto: {row['comment_text']}")
    print(f"Categorias tóxicas: {row['categorias_ativas']}")
    print("-" * 100)

Índice: 6
Texto: cock  suck before you piss around on my work
Categorias tóxicas: ['insult', 'obscene', 'severe_toxic', 'toxic']
----------------------------------------------------------------------------------------------------
Índice: 12
Texto: hey  what is it talk what is it  an exclusive group of some wp talibans who are good at destroying  self appointed purist who gang up any one who asks them questions abt their anti social and destructive  non  contribution at wp ask sityush to clean up his behavior than issue me nonsensical warnings 
Categorias tóxicas: ['toxic']
----------------------------------------------------------------------------------------------------
Índice: 16
Texto: bye  don t look  come or think of comming back  tosser 
Categorias tóxicas: ['toxic']
----------------------------------------------------------------------------------------------------
Índice: 42
Texto: you are  gay  or antisemmitian archangel white tiger  meow  greetingsh uh  there are two ways  w

In [228]:
# Selecione um exemplo tóxico para testar (altere o índice se quiser outro)
idx = 6

# Recupera dados
texto_original = df_embed.iloc[idx]['comment_text']
embedding_original = df_embed.iloc[idx]['embedding']
categorias_ativas = [cat for cat in categorias if df_embed.iloc[idx][cat] == 1]

print("Texto original:", texto_original)
print("Categorias tóxicas:", categorias_ativas)

# Aplica desintoxicação sequencial (simples soma dos vetores corretivos)
embedding_editado = embedding_original.copy()

for cat in categorias_ativas:
    vetor_toxico = vetores_categoria[cat]
    embedding_editado = embedding_editado - vetor_toxico + vetores_categoria['neutro']

# Converte para tensor
embedding_tensor = torch.tensor(embedding_editado).unsqueeze(0).to("cuda").to(torch.float16)

# Reconstrói texto
with torch.inference_mode():
    texto_desintoxicado = decoder.predict(embedding_tensor, target_lang="eng_Latn")

print("Texto desintoxicado:", texto_desintoxicado)

Texto original: cock  suck before you piss around on my work
Categorias tóxicas: ['insult', 'obscene', 'severe_toxic', 'toxic']
Texto desintoxicado: ['recommendation of chapter inclusion of text evaluation prior to the introduction of the report whereas it would be appropriate to refer to procedures before commenting on each article on the report in order to clarify whether it has been applied to other aspects of the report']


# Desintoxicação Inteligente

In [229]:
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
import numpy as np

# Embeddings e rótulos
X = np.stack(df_embed['embedding'].values)
Y = df_embed[categorias].astype(int)

# Split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Modelo base binário
base_model = LogisticRegression(max_iter=1000)
multi_model = MultiOutputClassifier(base_model)

# Treina
multi_model.fit(X_train, Y_train)

MultiOutputClassifier(estimator=LogisticRegression(max_iter=1000))

In [230]:
# Calcula os pesos (probabilidades) previstos pelo modelo de ML para o comentário de índice `idx`
probs = multi_model.predict_proba([X[idx]])  # usa o mesmo índice idx_toxico ou outro
pesos = [p[1] if len(p) > 1 else 0.0 for p in probs]  # pega a prob da classe 1 para cada categoria

In [231]:
# Garante que estamos usando o embedding original com 1024 dimensões
embedding_original = df_embed.loc[idx, 'embedding']  # idx definido anteriormente com o comentário tóxico
embedding_editado = embedding_original.copy()

# Aplica correção vetorial baseada nos pesos: (vetor_neutro - vetor_toxico)
for i, cat in enumerate(categorias):
    direcao_correta = vetores_categoria['neutro'] - vetores_categoria[cat]
    embedding_editado -= pesos[i] * direcao_correta

embedding_editado -= vetores_categoria['neutro']

# Converte para tensor com shape (1, 1024)
embedding_tensor = torch.tensor(embedding_editado, dtype=torch.float16).unsqueeze(0).to("cuda")
embedding_tensor_list = [embedding_tensor]  # formato exigido pelo decoder

print("embedding_original shape:", embedding_original.shape)
print("embedding_editado shape :", embedding_editado.shape)
print("embedding_tensor shape  :", embedding_tensor.shape)

embedding_original shape: (1024,)
embedding_editado shape : (1024,)
embedding_tensor shape  : torch.Size([1, 1024])


In [232]:
# Reconstrói texto
with torch.inference_mode():
    texto_desintoxicado = decoder.predict(embedding_tensor_list[0], target_lang="eng_Latn")

print("Texto original:", texto_original)
print(embedding_original)
print("Texto desintoxicado:", texto_desintoxicado[0])
print(embedding_editado)

Texto original: cock  suck before you piss around on my work
[ 0.001179  -0.00524    0.000661  ...  0.0009365 -0.00091    0.002306 ]
Texto desintoxicado: You suckle before you work.
[ 0.001625  -0.006546   0.0004134 ...  0.001793  -0.004616   0.003748 ]
